In [6]:
import os
import pandas as pd
import prompts

from cohere import Client
from dotenv import load_dotenv
from tqdm import tqdm


In [2]:

load_dotenv()
co = Client(os.getenv("COHERE_API_KEY"))
model_name = "command-r-plus-08-2024"  # Latest release of Command-R Plus


In [4]:

filepath = "../datasets/perturbed_solutions_0.csv"
data = pd.read_csv(filepath)
data.head()


,id,question,solution,stepped,perturbed,step,type,trace
0,2,"In $\triangle ABC$, the lengths of the sides o...","Since $\cos \frac{C}{2} = \frac{\sqrt{5}}{3}$,...",Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,2,Arithmetic sign error,The sign in the second term of the equation in...
1,3,Given that $P$ is any point on the circle $C$:...,The distance $d$ between the center of the cir...,Step 1: The distance $d$ between the center of...,Step 1: The distance $d$ between the center of...,2,Confusing variables,The variable $P$ was mistakenly used as $Q$ in...
2,4,Factorize: $x^{3}-6x^{2}+9x=\_\_\_\_\_\_.$,To factorize the given expression $x^{3}-6x^{2...,Step 1: To factorize the given expression $x^{...,Step 1: To factorize the given expression $x^{...,2,Algebraic manipulation error,The perturbation was applied by incorrectly fa...
3,5,Given that the sum of the first $n$ terms of a...,"Since $S\_n=2^{n}a\_n-1$,\nwe have $S_{n-1}=2^...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...",2,Arithmetic sign error,The subtraction operation in Step 2 was incorr...
4,6,"Given the function $f(x)=x\ln x$, if $0 \lt x_...","To analyze each option step by step, we start ...","Step 1: To analyze each option step by step, w...","Step 1: To analyze each option step by step, w...",3,Dropped negative sign,The derivative of $g(x)$ is calculated as $g'(...


In [5]:
def print_row_information(row: pd.Series):
    print("~~~Row Information~~~")
    print(f"ID: {row['id']}")
    print(f"Question: {row['question']}")
    print(f"Stepped Answer: {row['stepped']}")
    print(f"Type: {row['type']}")
    print(f"Step: {row['step']}")
    print(f"Trace: {row['trace']}")
    print(f"Perturbed: {row['perturbed']}")
    print("~~~End of Row Information~~~")


def get_row_completion(row: pd.Series):
    user_turn = prompts.COMPLETION_PROMPT_V2_USER.format(question=row["question"])
    assistant_turn = prompts.COMPLETION_PROMPT_V2_ASSISTANT.format(perturbed_reasoning=row["perturbed"])
    print(f"\n\n User propmt: {user_turn}\n\n Assistant prompt: {assistant_turn}\n\n")
    completion = co.chat(
        message=prompts.RAW_COMPLETION_TEMPLATE.format(user_turn=user_turn, assitant_turn=assistant_turn),
        raw_prompting=True,
    )
    print(f"\n\n Completion: {completion.text}\n\n")
    return completion.text


In [9]:
completion_df = data[0:10]

completions = []

for index, row in tqdm(completion_df.iterrows(), total=len(data), desc="Processing rows"):
    print("------------------Row------------------")
    print_row_information(row)
    completions.append(get_row_completion(row))
    print("------------------End of Row------------------")

completion_df["completion"] = completions

Processing rows:   0%|          | 0/47 [00:00<?, ?it/s]

------------------Row------------------
~~~Row Information~~~
ID: 2
Question: In $\triangle ABC$, the lengths of the sides opposite to angles $A$, $B$, and $C$ are $a$, $b$, and $c$ respectively. Given that $\cos \frac{C}{2} = \frac{\sqrt{5}}{3}$ and $a \cos B + b \cos A = 2$, find the maximum area of $\triangle ABC$.
Stepped Answer: Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{5}}{3}$, we have $\cos C = 2\cos^2 \frac{C}{2} - 1 = 2 \left(\frac{\sqrt{5}}{3}\right)^2 - 1 = \frac{1}{9}$.

Step 2: Using the cosine law, we have $a \cos B + b \cos A = 2$ can be written as $a \frac{a^2 + c^2 - b^2}{2ac} + b \frac{c^2 + b^2 - a^2}{2bc} = 2$.

Step 3: Simplifying the equation, we obtain $c = 2$.

Step 4: Now, we have $4 = a^2 + b^2 - 2ab \cos C \geq 2ab - 2ab \frac{1}{9} = \frac{16}{9}ab$, which implies $ab \leq \frac{9}{4}$. The equality holds when $a = b = \frac{3}{2}$.

Step 5: Using the sine law, we have $\sin C = \sqrt{1 - \cos^2 C} = \sqrt{1 - \left(\frac{1}{9}\right)^2} = \frac{4\sqrt{5

Processing rows:   2%|▏         | 1/47 [00:11<08:58, 11.70s/it]



 Completion: 
Step 3: Simplifying the equation in Step 2, we get $a^3b - ab^3 + a^2c^2 - b^2c^2 - 2abc^2 = 36ac^2$.

Step 4: Note that $a^3b - ab^3 = a^2b(a-b) \leq \frac{1}{4} (a^2 + b^2)^2 = \frac{1}{4} (2c^2 + 2ab)^2 = \frac{1}{4} (2c^2 + 2a^2 + 2b^2 - a^2 - b^2)^2 = \frac{1}{4} (a^2 + b^2 + c^2)^2$.

Step 5: From Step 3 and Step 4, we have $\frac{1}{4} (a^2 + b^2 + c^2)^2 + a^2c^2 - b^2c^2 - 2abc^2 \geq 36ac^2$, which simplifies to $(a - b)^2 (a^2 + b^2 + c^2) \geq 1296$.

Step 6: Since $(a - b)^2 \geq 0$, we have $a^2 + b^2 + c^2 \geq 36$.

Step 7: By Heron's formula, the area of $\triangle ABC$ is $S = \sqrt{s(s-a)(s-b)(s-c)}$, where $s = \frac{a+b+c}{2}$.

Step 8: The maximum area occurs when $a = b = c$, so $S_{\max} = \sqrt{s(s-a)(s-b)(s-c)} = \sqrt{\frac{3a}{2}\left(\frac{a}{2}\right)\left(\frac{a}{2}\right)\left(\frac{a}{2}\right)} = \frac{a^2\sqrt{3}}{8}$.

Step 9: From Step 6, we have $a^2 \geq 36$, so $S_{\max} = \frac{a^2\sqrt{3}}{8} \geq \frac{36\sqrt{3}}{8}$.

<answe

Processing rows:   4%|▍         | 2/47 [00:19<07:02,  9.39s/it]

...

Processing rows:  19%|█▉        | 9/47 [01:32<06:00,  9.49s/it]



 Completion: 
Step 3: Let $f'(x)=0$, we get $ax^2+(1-2a)x+a-1=0$

Step 4: Since $a\geqslant 0$, we have $x_1=\frac{-1+\sqrt{1+4a}}{2a}\geqslant 0$, $x_2=\frac{-1-\sqrt{1+4a}}{2a}\leqslant 0$

Step 5: Since $f''(x)=e^{-x}[a(x-1)^2-1]$, we have $f''(x_1)=e^{-x_1}[a(x_1-1)^2-1]\leqslant 0$

Step 6: Therefore, $x_1$ is the maximum point of the function $f(x)$

Step 7: Since $f(x_1)=\frac{3}{e}$, we have $\frac{3}{e}=f(x_1)=\frac{a(1+4a)}{4ae}$, then $a=0$ or $a=1$

Step 8: If $a=0$, then $f(x)=e^{-x}$, and the maximum value of $f(x)$ is $1$

Step 9: If $a=1$, then $f(x)=(x^2+x+1)e^{-x}$, and the maximum value of $f(x)$ is $\frac{3}{e}$

Step 10: Therefore, the value of the real number $a$ is $\boxed{<1>}$

Step 11: (2) Since $a\leqslant 0$, we have $f(x)\leqslant 0$

Step 12: Since $b\ln(x+1)\geqslant 0$, we have $b\geqslant 0$

Step 13: Therefore, the range of the real number $b$ is $\boxed{<0,+\infty)>$}


------------------End of Row------------------
------------------Row------------

Processing rows:  21%|██▏       | 10/47 [01:40<06:13, 10.10s/it]



 Completion: 
Step 5: (2) From $(4\overrightarrow{a}-\overrightarrow{b}) \perp (2\overrightarrow{a}+\overrightarrow{b})$, Step 6: We get $(4\overrightarrow{a}-\overrightarrow{b})\cdot(2\overrightarrow{a}+\overrightarrow{b})=0$, Step 7: Expanding, we get $8\overrightarrow{a}\cdot\overrightarrow{a}+4\overrightarrow{a}\cdot\overrightarrow{b}-2\overrightarrow{a}\cdot\overrightarrow{b}-\overrightarrow{b}\cdot\overrightarrow{b}=0$, Step 8: Simplifying, we get $8|\overrightarrow{a}|^{2}+2\overrightarrow{a}\cdot\overrightarrow{b}=0$, Step 9: Since $|\overrightarrow{a}|^{2}=1^{2}+2^{2}=5$, Step 10: We get $40+2\overrightarrow{a}\cdot\overrightarrow{b}=0$, Step 11: Solving for $\overrightarrow{a}\cdot\overrightarrow{b}$, we find $\overrightarrow{a}\cdot\overrightarrow{b}=-20$, Step 12: Using the formula for the cosine of the angle between two vectors, Step 13: We have $\cos\theta=\frac{\overrightarrow{a}\cdot\overrightarrow{b}}{|\overrightarrow{a}||\overrightarrow{b}|}$, Step 14: Substituting 


/tmp/ipykernel_1164289/211227130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completion_df["completion"] = completions


In [14]:
completion_df.head()
# Completion_df columns include id, question, solution, stepped, perturbed, step, type, trace, completion
# All columns are int besides id and step

,id,question,solution,stepped,perturbed,step,type,trace,completion
0,2,"In $\triangle ABC$, the lengths of the sides o...","Since $\cos \frac{C}{2} = \frac{\sqrt{5}}{3}$,...",Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,2,Arithmetic sign error,The sign in the second term of the equation in...,"\nStep 3: Simplifying the equation in Step 2, ..."
1,3,Given that $P$ is any point on the circle $C$:...,The distance $d$ between the center of the cir...,Step 1: The distance $d$ between the center of...,Step 1: The distance $d$ between the center of...,2,Confusing variables,The variable $P$ was mistakenly used as $Q$ in...,"\nStep 3: Let the coordinates of $P$ be $(x,y)..."
2,4,Factorize: $x^{3}-6x^{2}+9x=\_\_\_\_\_\_.$,To factorize the given expression $x^{3}-6x^{2...,Step 1: To factorize the given expression $x^{...,Step 1: To factorize the given expression $x^{...,2,Algebraic manipulation error,The perturbation was applied by incorrectly fa...,"\nStep 3: Now, we focus on the expression insi..."
3,5,Given that the sum of the first $n$ terms of a...,"Since $S\_n=2^{n}a\_n-1$,\nwe have $S_{n-1}=2^...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...",2,Arithmetic sign error,The subtraction operation in Step 2 was incorr...,\nStep 3: Dividing both sides by $2^{n-1}a_{n+...
4,6,"Given the function $f(x)=x\ln x$, if $0 \lt x_...","To analyze each option step by step, we start ...","Step 1: To analyze each option step by step, w...","Step 1: To analyze each option step by step, w...",3,Dropped negative sign,The derivative of $g(x)$ is calculated as $g'(...,"2. Take the derivative of $g(x)$, which is $g'..."


In [13]:
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
completion_df.to_csv(f"../datasets/completions/solutions_perturbed_0_completions_command_r-{now}.csv", index=False)